## <B>Normalize Gaze Coordinate Data</B>

In [ ]:
# Load the libraries you need
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

name = "37"

movies = ["기생충1", "기생충2", "낭닥1", "낭닥2", "코1", "코2",  "마녀1", "마녀2", "레버1", "레버2"]

for movie in movies:
    # Importing data files
    data = pd.read_csv(f'{name} {movie}.csv')
    
    # Check column names and remove spaces
    data.columns = data.columns.str.strip()  # Remove unnecessary spaces in column names
    
    # Check the X and Y columns
    print("원본 데이터:")
    print(data.head())
    
    # Normalize using Min-Max Scaler (range: -1 to 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    
    # Normalize X and Y
    data[['X', 'Y']] = scaler.fit_transform(data[['X', 'Y']])
    
    # Check the results
    print("\nNormalized data:")
    print(data.head())
    
    # Save the normalized data as a new CSV file
    data.to_csv(f'{name} {movie}(정규화).csv', index=False)

## <B>Presence of eyes in blurred areas of experimental footage O/X</B>

In [ ]:
import pandas as pd
import warnings

# Set to ignore warnings
warnings.filterwarnings("ignore")

# Define OX analytics and integration processing functions
def process_and_analyze(name, movie, box_file):
    try:
        # Read data
        points_df = pd.read_csv(f'{name} {movie}(정규화).csv')  # Normalized point data
        boxes_df = pd.read_csv(box_file)  # Box data

        # Convert and reset timestamps
        points_df['Timestamp (Current Time)'] = pd.to_datetime(points_df['Timestamp (Current Time)'], errors='coerce')
        points_df.dropna(subset=['Timestamp (Current Time)'], inplace=True)  # Delete NaT

        # Setting the base time and converting to seconds
        base_time = points_df['Timestamp (Current Time)'].iloc[0]
        points_df['Time (seconds)'] = (points_df['Timestamp (Current Time)'] - base_time).dt.total_seconds()

        # Rounding and deduplicating time values
        points_df['Time (seconds)'] = points_df['Time (seconds)'].round(1)
        boxes_df['Time (seconds)'] = boxes_df['Time (seconds)'].round(1)
        points_df = points_df.drop_duplicates(subset='Time (seconds)', keep='first')
        boxes_df = boxes_df.drop_duplicates(subset='Time (seconds)', keep='first')

        # Merge by time
        merged_df = pd.merge(boxes_df, points_df, how='left', on='Time (seconds)')
        merged_df.to_csv(f'{name}_{movie}_merged.csv', index=False)  # Save the merge results

        # Calculate OX results
        def point_box_with_margin(row):
            expanded_x1 = row['x1'] - 0.25
            expanded_y1 = row['y1'] - 0.25
            expanded_x2 = row['x2'] + 0.25
            expanded_y2 = row['y2'] + 0.25
            return "O" if expanded_x1 <= row['X'] <= expanded_x2 and expanded_y1 <= row['Y'] <= expanded_y2 else "X"

        merged_df['Result'] = merged_df.apply(point_box_with_margin, axis=1)

        # Calculate and output the O ratio
        o_count = merged_df['Result'].value_counts().get('O', 0)
        total_count = len(merged_df)
        o_ratio = o_count / total_count if total_count > 0 else 0

        print(f'"{movie}" Analysis results:')
        print(f' Number of "O": {o_count}')
        print(f' Percentage of "O": {o_ratio:.2%}')

        # Save Final Result
        merged_df.to_csv(f'{name} {movie}_result.csv', index=False)

    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"Error occurred: {e}")

# Example run
name = "37"
movie_box_map = {
    "기생충1": "영화1 보정.csv",
    "기생충2": "영화1 보정.csv",
    "낭닥1": "영화2 보정.csv",
    "낭닥2": "영화2 보정.csv",
    "코1": "영화3 보정.csv",
    "코2": "영화3 보정.csv",
    "마녀1": "영화4 보정.csv",
    "마녀2": "영화4 보정.csv",
    "레버1": "영화5 보정.csv",
    "레버2": "영화5 보정.csv",
}

for movie, box_file in movie_box_map.items():
    process_and_analyze(name, movie, box_file)